# Apply to Other Datasets

## [Task 3] Create a Model of Iris Dataset with All the 3 Kinds of Objective Variables

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Read dataset

dataset_path = "Iris.csv"
df = pd.read_csv(dataset_path)

In [3]:
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

y = np.array(y)
X = np.array(X)

In [4]:
# Change labels to values

y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2

y = y.astype(np.int)[:, np.newaxis]

In [5]:
# One hot encoding

# Initialize
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Fit
y_one_hot = ohe.fit_transform(y)

In [6]:
# Split the dataframe into train and test datasets

X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=0)

In [7]:
# Split the train dataset into train and validation datasets again

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [8]:
class GetMiniBatch:
    """
    Iterator to get mini-batches
    
    Parameters
    ----------
    X : ndarray, shape (n_samples, n_features)
        Train data
    
    y : ndarray, shape (n_samples, 1)
        Correct values
    
    batch_size : int
        Size of a batch
    
    seed : int
        Seed of random values of NumPy
    """
    
    def __init__(self, X, y, batch_size=10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    
    
    def __len__(self):
        return self._stop
    
    
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        
        return self.X[p0:p1], self.y[p0:p1]
    
    
    def __iter__(self):
        self._counter = 0
        
        return self
    
    
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        
        self._counter += 1
        
        return self.X[p0:p1], self.y[p0:p1]

In [9]:
# Set hyperparameters

learning_rate = 0.01
batch_size = 10
num_epochs = 10

In [10]:
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3

In [11]:
# Decide shapes of arguments given to a dataflow graph

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

In [12]:
# Mini-batch interator of train dataset

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

In [13]:
def example_net(x):
    """
    Simple neural network of 3 layers
    """
    
    # Declaration of weights and biases
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    
    # 1st layer
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    # 2nd layer
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    # 3rd layer
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3']
    
    return layer_output

In [14]:
# Read network construction

logits = example_net(X)

In [15]:
# Objective function

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))

In [16]:
# Optimization method

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [17]:
# Prediction results

correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(tf.nn.softmax(logits), axis=1))

In [18]:
# Compute index values

accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [19]:
# Initialize variables

init = tf.global_variables_initializer()

In [20]:
# Run the dataflow graph

with tf.Session() as sess:
    sess.run(init)
    
    # Loop per epoch
    for epoch in range(num_epochs):
        # Loop per mini-batch
        for mini_batch_x, mini_batch_y in get_mini_batch_train:
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
        
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 30.5082, val_loss : 9.3375, acc : 0.167, val_acc : 0.375
Epoch 1, loss : 0.6781, val_loss : 0.3897, acc : 0.833, val_acc : 0.958
Epoch 2, loss : 0.0000, val_loss : 1.3292, acc : 1.000, val_acc : 0.917
Epoch 3, loss : 0.0001, val_loss : 2.0836, acc : 1.000, val_acc : 0.917
Epoch 4, loss : 3.9593, val_loss : 8.1911, acc : 0.500, val_acc : 0.708
Epoch 5, loss : 0.0000, val_loss : 0.8439, acc : 1.000, val_acc : 0.875
Epoch 6, loss : 0.0000, val_loss : 0.6743, acc : 1.000, val_acc : 0.917
Epoch 7, loss : 0.0000, val_loss : 0.9631, acc : 1.000, val_acc : 0.958
Epoch 8, loss : 0.0000, val_loss : 1.6742, acc : 1.000, val_acc : 0.917
Epoch 9, loss : 0.6526, val_loss : 4.6657, acc : 0.833, val_acc : 0.833
test_acc : 0.767
